In [1]:
import psycopg2
import pandas as pd
import numpy as np
from configparser import ConfigParser
from config import config
from connect import connect
from query import query
from create_table import table_from_df
from sqlalchemy import create_engine, engine_from_config
from sqlalchemy.engine import URL


In [2]:
# currently using dummy excel files to read in as DataFrames

listings_df = pd.read_excel(r'listings_test.xlsx')
sold_df = pd.read_excel(r'sold_test.xlsx')

In [3]:
# create new database tables from listings and sold DataFrames

table_from_df(listings_df, 'listings')

Connecting to the PostgreSQL database...
Table listings created
Database connection closed.


In [4]:
table_from_df(sold_df, 'sold')

Connecting to the PostgreSQL database...
Table sold created
Database connection closed.


In [11]:
# Extract sold table to train ML model on

sold_export = query('SELECT * FROM sold', 'sold', to_df = True)

Connecting to the PostgreSQL database...
Query executed.
Database connection closed.


In [12]:
sold_export

,ID,Address,Property Type,Bedrooms,Bathrooms,Sold Date,Sold Price
0,SoldA,Sold1,Apartment,1.0,1.0,2021-12-01,150000
1,SoldB,Sold2,None,7.0,5.0,1998-07-05,600000
2,SoldC,Sold3,Terrace,NaN,NaN,2012-04-16,150000
3,SoldD,Sold4,Detached,4.0,NaN,NaT,475000
4,SoldE,Sold5,Detached,NaN,4.0,NaT,532000
5,SoldF,Sold6,Test,1.0,1.0,2001-01-01,101010


In [5]:
# Extract listings table to make predictions on

listings_export = query('SELECT * FROM listings', 'listings', to_df = True)

Connecting to the PostgreSQL database...
Query executed.
Database connection closed.


In [6]:
listings_export

,ID,Address,Property Type,Bedrooms,Bathrooms,Listed Date,Listed Price
0,ListedA,Test1,Detached,4,3,2023-02-15,500000
1,ListedB,Test2,Terrace,2,1,2022-12-22,230000
2,ListedC,Test3,Apartment,1,1,2023-04-11,185000
3,ListedD,Test4,Detached,3,2,2023-01-01,350000
4,ListedE,Test5,Semi-detached,3,2,2021-02-10,320000


In [7]:
# Imagine that the export has been put through the ML Pipeline and a prediction has been made.

listings_export['Prediction'] = np.random.normal(listings_export['Listed Price'], 0.1*listings_export['Listed Price']).round(2)

In [8]:
listings_export['Value Delta'] = (listings_export['Prediction'] - listings_export['Listed Price']).round(2)

In [9]:
listings_export_trim = listings_export[['ID', 'Prediction', 'Value Delta']]

In [10]:
# Feed predictions back into database as a new table.

table_from_df(listings_export_trim,'predictions')

Connecting to the PostgreSQL database...
Table predictions created
Database connection closed.
